# Context

This is what is investigated here

# Imports

In [ ]:
from skimage import io
import skimage
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, uniform_filter
import pickle

In [ ]:
import imageio
from pathlib import Path
from matplotlib.pyplot import show
from argparse import ArgumentParser

from pyoptflow.plots import compareGraphs

In [ ]:
from PIL import Image
import os
from scipy.signal import argrelextrema
from skimage import exposure

In [ ]:
import matplotlib
import matplotlib.animation
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128

### Import our custom utility methods

In [ ]:
!ls ../utils/

In [ ]:
import sys
%reload_ext autoreload
%autoreload 2
sys.path.append('..')

from utils.visualization_tools import *
import utils.visualization_tools
from utils.data_transformations import *
import utils.data_transformations
from utils.diverse import *
import utils.diverse

import utils.levelset_flow
from utils.levelset_flow import *

The following modules are available

In [ ]:
print_module_methods(utils.diverse)

In [ ]:
print_module_methods(utils.visualization_tools)

In [ ]:
print_module_methods(utils.data_transformations)

In [ ]:
from pathlib import Path
source_folder = os.path.join(Path(os.getcwd()).parent, "source_data")

files = []
files.append(os.path.join(source_folder,"runstart16_X1.tif"))
files.append(os.path.join(source_folder,"runstart16_X2.tif"))

filepath = files[0]

In [ ]:
mask = np.array(Image.open(os.path.join(source_folder,"mask_runstart16_X_left.png")))==0
mask = mask [:,:,0]
mask = ~mask

In [ ]:
frames = np.array(skimage.io.imread(os.path.join(filepath)), dtype=np.double)[:1000]
mean = np.mean(frames,axis=0)#pixelwise mean
difference = framewise_difference(frames, mean, bigdata=True)
mean = None

In [ ]:
frames = difference.copy()
frames = remove_frequency_from_pixel_vectors(frames,15,20)

In [ ]:
frames = gaussian_filter(frames,2)

In [ ]:
expected = np.load("10_bins_expected_images_polynomial.npy")

# Remove the expected image from each frame

In [ ]:
expected = np.load("10_bins_expected_images_polynomial.npy")

In [ ]:
frames = substract_expected(frames, expected, True)

In [ ]:
frames -= np.min(frames)

In [ ]:
frames = apply_mask(frames, mask)

In [ ]:
frames = normalize(frames)

In [ ]:
plt.imshow(frames[0])

In [ ]:
#frames = substract_pixel_min(frames)

In [ ]:
%%capture
ani = show_video(normalize(frames[540:]), normalize(difference[540:]), n_frames=100, vmin = .1, vmax=.5)

In [ ]:
HTML(ani)

In [ ]:
from scipy.ndimage import binary_erosion
from scipy.ndimage import binary_dilation

In [ ]:
#ls = levelsets(frames[565:800], .1, .4, 8)

In [ ]:
ls = levelsets(frames, .1, .5, 20)

In [ ]:
test_array = ls[580][780:1400,:]

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))
ax.imshow(test_array)

In [ ]:
n_iterations = 8
arr = binary_erosion(binary_dilation(test_array, iterations=n_iterations),iterations=n_iterations)
plt.imshow(arr)

In [ ]:
closed = framewise_closing(ls[:], n_iterations=12, smoothing=4, post_smoothing=4)

In [ ]:
n_plots = 50
start = 550
fig, ax = plt.subplots(1,n_plots, figsize=(30,30))
for i in range(0,n_plots):
    ax[i].imshow(ls[start+i])
    ax[i].axis("off")

In [ ]:
n_plots = 50
fig, ax = plt.subplots(1,n_plots, figsize=(30,30))
for i in range(0,n_plots):
    ax[i].imshow(ls[start+i+80])
    ax[i].axis("off")

In [ ]:
%%capture
ani = show_video(ls[start:],closed[start:], n_frames = 100)

In [ ]:
HTML(ani)

In [ ]:
n_levels = 20
ls1 = closed.reshape(len(closed),n_levels, closed[0].shape[0]//n_levels,closed[0].shape[1])

In [ ]:
tensor_of_interest = normalize(np.mean(ls1[570:580,0:3,:,:],axis=(1)))

In [ ]:
plt.imshow(tensor_of_interest[1])

In [ ]:
assert np.all(np.mean(grayscale_to_booleans(tensor_of_interest[0], 12),axis=0)-tensor_of_interest[0] == 0.0)

In [ ]:
ys, xs, bg = levelset_flow(tensor_of_interest, 2, stepsize=1)

In [ ]:
bg = normalize(bg)

In [ ]:
#ys, xs = horn_schunck(gaussian_filter(tensor_of_interest.astype(np.float32),2))

In [ ]:
fig, ax = display_combined(dy[0],dx[0], tensor_of_interest[1])
frames = len(ys)

def animate(i):
    print(".", end ="")    
    display_combined(ys[i],xs[i], bg[i], quivstep=1,  fig=fig, ax=ax, scale=1)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=frames).to_jshtml()

In [ ]:
HTML(ani)

In [ ]:
tensor_of_interest = normalize(np.mean(ls1[560:900,0:1,:,:],axis=(1)))

In [ ]:
tensor_of_interest = normalize(gaussian_filter(tensor_of_interest[:],[1,10,10]))>.1#Smoothen outline

In [ ]:
x_comp, y_comp = horn_schunck(normalize(gaussian_filter(tensor_of_interest.astype(np.float32),[0,5,10])), 305)

In [ ]:
%%capture
ani = show_video(normalize(gaussian_filter(tensor_of_interest.astype(np.float32),[0,5,10])),n_frames=100)

In [ ]:
HTML(ani)

In [ ]:
def dense_to_contour_vectors(tensor, y_comp, x_comp):
    y_out = np.ndarray(y_comp.shape)
    x_out = np.ndarray(x_comp.shape)
    y_out.fill(np.nan)
    x_out.fill(np.nan)

    for i, [y,x,frame] in enumerate(zip(y_comp, x_comp, tensor)):
        values = set(np.array(frame.flatten()))
        values.remove(0)
        for v in values:  
            frame1 = frame == v
            eroded = frame1.astype(np.int32)-binary_erosion(frame1, iterations=1).astype(np.int32)
            y_out[i][eroded==1] = y[eroded==1]
            x_out[i][eroded==1] = x[eroded==1]
    return y_out, x_out

In [ ]:
print(1)

In [ ]:
x_comp, y_comp = dense_to_contour_vectors(tensor_of_interest[:], x_comp[:], y_comp[:])

In [ ]:
#x_comp[:,mask] = np.nan
#y_comp[:,mask] = np.nan

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], tensor_of_interest[1])
time_text = plt.figtext(0.5, 0.01, "time " + str(i), ha="center", fontsize=18)

def animate(i):
    global time_text
    print(".", end ="")    
    display_combined(y_comp[i],x_comp[i], tensor_of_interest[i], fig=fig, ax=ax, scale=50, quivstep = 1)
    time_text.set_text('time = %.1d' % i)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=300)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,10))
ax.plot(np.nansum(x_comp, axis=(1,2))[:], c="lightgray", label="x component")
ax.plot(np.nansum(-y_comp, axis=(1,2))[:],c="darkblue", label = "y component")
ax.axvline(78, c = "gray")
ax.set_xlabel("Frame")
ax.set_ylabel("Mean component of vectors [upwards/rightwards]")

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], tensor_of_interest[1])

def animate(i):
    print(".", end ="")    
    display_combined(y_comp[i],x_comp[i], tensor_of_interest[i], fig=fig, ax=ax, scale=50, quivstep = 1)
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=300)

# Towards histograms of motion

In [ ]:
print("The upper half circle is negative")
print("left  = -180 deg")
print("up    =  -90 deg")
print("right =    0 deg")
print("down  =  +90 deg")
print("left  =  180 deg")

In [ ]:
plot_vector_as_field(y=-0.001, x=-1)

In [ ]:
plot_vector_as_field(y=0,x=1)

# Expectation and histogram

We would want to have something to capture the upward trend.

In [ ]:
display_combined(vois_y,vois_x,vois_y, quivstep=1, scale=10)

For position 10 we see a positive value for the average y component (overall trend of motion)

In [ ]:
vois_x = x_comp[10]
vois_y = y_comp[10]
plt.plot(np.nansum(-y_comp, axis=(1,2))[:])
plt.axvline(10, c = "gray")

In [ ]:
plt.plot(np.nansum(x_comp, axis=(1,2))[:])
plt.axvline(10, c = "gray")

For a random location at the upper left outline arctan2 indicates what is expected (sanity check)

In [ ]:
plot_vector_as_field(vois_x[109,62],vois_y[109,62])

# The average vector indicates upward motion

In [ ]:
plot_vector_as_field(np.nanmean(vois_y),np.nanmean(vois_x))

In [ ]:
x = vois_x[~np.isnan(vois_x)]
y = vois_y[~np.isnan(vois_y)]

mask = ~np.logical_and(x==0,y==0)
x = x[mask]
y = y[mask]
mag = np.sqrt(x**2, y**2)
dir = np.rad2deg(np.arctan2(y,x))

In [ ]:
weighted_hist = np.histogram(dir, weights= mag**4 ,bins= 36)
hist = np.histogram(dir, bins= 36)

In [ ]:
plt.plot(hist[1][:-1],hist[0])
#plt.plot(weighted_hist[1][:-1],weighted_hist[0])
#plt.plot(weighted_hist[1][:-1],weighted_hist[0]/hist[0])

--> Weighted histograms do not have a peak at the expected position (average direction)

In [ ]:
def histograms_of_motion(x_comp, y_comp, bins = 36, weighted = True):
    x_comp = x_comp.reshape(x_comp.shape[0],x_comp.shape[1]*x_comp.shape[2])
    y_comp = y_comp.reshape(y_comp.shape[0],y_comp.shape[1]*y_comp.shape[2])
    
    histograms = []
    
    for x, y in zip(x_comp, y_comp):
        x = x[~np.isnan(x)]
        y = y[~np.isnan(y)]
        
        mask = ~np.logical_and(x==0,y==0)
        x = x[mask]
        y = y[mask]
        assert len(x) == len(y)
        mag = np.sqrt(x**2, y**2)
        dir = np.rad2deg(np.arctan2(y,x))
        if weighted:
            hist = np.histogram(dir, weights = mag, bins=bins, range=(-180,180))[0]
        else:
            hist = np.histogram(dir, bins=bins, range=(0,360))[0]
        #if np.sum(hist) !=0:
        #    histograms.append(normalize(hist))
        #else:
        histograms.append(hist)
    return np.array(histograms).T

In [ ]:
total_y  = -np.nansum(y_comp,axis=(1,2))
total_y /= np.max([np.max(total_y),np.min(total_y)])
#plt.plot(np.nansum(x_comp,axis=(1,2)))

In [ ]:
hists = histograms_of_motion(x_comp, y_comp, bins=360)

In [ ]:
test_y = np.array([[[.01]],[[.01]],[[-1]],[[1]]], dtype=np.float32)
test_x = np.array([[[-1]],[[1]],[[.1]],[[.1]]], dtype=np.float32)

In [ ]:
hists = histograms_of_motion(x_comp, y_comp, bins=36)

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,15))
ax.set_yticks([0,9,18,27,35])
ax.set_yticklabels(["left","up","right","down","left"])

#ax.plot(np.nansum(x_comp,axis=(1,2)))

ax.imshow(np.log(hists[:,:]+1), aspect=1)

# Correct vector fields

In [ ]:
n_levels = 20
ls = levelsets(frames, .1, .5, n_levels)
closed = framewise_closing(ls[:], n_iterations=12, smoothing=4, post_smoothing=4)
ls1 = closed.reshape(len(closed),n_levels, closed[0].shape[0]//n_levels,closed[0].shape[1])
reassembled = normalize(np.mean(ls1[560:900,:,:,:],axis=(1)))
smooth_reassembled = gaussian_filter(reassembled,[0,10,10])
y_comp_expected, x_comp_expected = expected_flow(smooth_reassembled, 202)

In [ ]:
%%capture
ani = show_video(gaussian_filter(reassembled,10),n_frames=200)

In [ ]:
HTML(ani)

In [ ]:
x_comp, y_comp = horn_schunck(gaussian_filter(reassembled,[0,10,10]), 202)

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], reassembled[1], quivstep = 10)
time_text = plt.figtext(0.5, 0.01, "time " + str(0), ha="center", fontsize=18)

def animate(i):
    global time_text
    print(".", end ="")    
    display_combined(y_comp[i],x_comp[i], reassembled[i], fig=fig, ax=ax, scale=500, quivstep = 5)
    time_text.set_text('time = %.1d' % i)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=200)

In [ ]:
plt.plot(np.mean(reassembled, axis=(1,2)))

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
y_comp_expected, x_comp_expected = expected_flow(smooth_reassembled, 202)

In [ ]:
current_frame = 11
#display_combined(y_comp[current_frame],x_comp[current_frame], reassembled[current_frame], quivstep=5, scale=500)

In [ ]:
y_comp_corrected, x_comp_corrected = correct_vector_fields(y_comp, y_comp_expected, x_comp, x_comp_expected, logscale = True)

In [ ]:
fig, ax = plt.subplots(2, figsize=(10,10))

ax[0].axhline(0, c="gray")
ax[0].plot(np.nanmean(-np.array(y_comp_corrected), axis=(1,2)))
ax[0].set_xlabel("Frame")
ax[0].set_ylabel("Motion score [up/down]")

ax[1].axhline(0, c="gray")
ax[1].plot(np.nanmean(-np.array(x_comp_corrected), axis=(1,2)))
ax[1].set_xlabel("Frame")
ax[1].set_ylabel("Motion score [left/right]")

In [ ]:
y_comp_corrected, x_comp_corrected = correct_vector_fields(y_comp, y_comp_expected, pre_log_factor = 5000)

In [ ]:
y_comp_corrected = gaussian_filter(y_comp_corrected*100,1)
x_comp_corrected = gaussian_filter(x_comp_corrected*100,1)

In [ ]:
%%capture
i = 0
fig, ax = display_combined(x_comp_corrected[0],y_comp_corrected[0], reassembled[1], quivstep = 10)
time_text = plt.figtext(0.5, 0.01, "time " + str(0), ha="center", fontsize=18)

def animate(i):
    global time_text
    print(".", end ="")    
    display_combined(y_comp_corrected[i],x_comp_corrected[i],reassembled[i], fig=fig, ax=ax, scale=3, quivstep = 5, head_width=2.5)
    time_text.set_text('time = %.1d' % i)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=50)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
from pynhhd import nHHD

def helmholtz_decomposition(y_comp, x_comp):
    vfield = np.array([y_comp,x_comp])
    vfield = np.einsum("ijk->kij",vfield)
    vfield = np.einsum("ijk->kij",vfield)
    dims = (vfield.shape[0],vfield.shape[1])
    nhhd = nHHD(grid=dims, spacings=(0.1,0.1))
    nhhd.decompose(vfield)
    return nhhd.r, nhhd.d, nhhd.h

In [ ]:
y_comp[np.isnan(y_comp)] = 0

In [ ]:
x_comp[np.isnan(x_comp)] = 0

In [ ]:
r, d, h = helmholtz_decomposition(y_comp[10], x_comp[10])

In [ ]:
display_combined(r[:,:,1],r[:,:,0],tensor_of_interest[10],quivstep=3,scale=100, head_width = 2)
plt.show()

In [ ]:
i_frame = 10
display_combined(x_comp[i_frame],y_comp[i_frame],np.sqrt(y_comp[3]**2+x_comp[3]**2)*100, quivstep=3)

In [ ]:
%%capture
lengths = np.sqrt(x_comp**2+y_comp**2)
ani = show_video(lengths*10, n_frames=90)

In [ ]:
%%capture
ani = show_video(normalize(max_filtered[550:]), normalize(difference[550:]), n_frames=50, vmin = .1, vmax=.3)

In [ ]:
HTML(ani)

In [ ]:
%%capture
ani = show_video(normalize(gaussian_filter(interpolated,2)), n_frames=199, vmin = .1, vmax=.5)

In [ ]:
HTML(ani)

In [ ]:
x_comp, y_comp = horn_schunck(normalize(gaussian_filter(interpolated,2)), 150)

In [ ]:
np.save("test_vector_field.npy",np.array([x_comp[75],y_comp[75]]))

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], interpolated[1])
start = 70
n_frames = 10

def animate(i):
    global start
    i += start
    print(".", end ="")    
    display_combined(y_comp[i],x_comp[i], interpolated[i+1], fig=fig, ax=ax, scale=1000)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=n_frames)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
#from scipy.ndimage.measurements import center_of_mass

In [ ]:
left_hemisphere = difference[:,:,:160]
right_hemisphere = difference[:,:,160:]
cm_left = np.array([center_of_mass(f) for f in left_hemisphere])
cm_right = np.array([center_of_mass(f) for f in right_hemisphere])

fig, ax = plt.subplots(3)
fig.suptitle("Center of mass (contrast to mean image)")
ax[0].plot(np.sum(difference, axis=(1,2)))

ax[1].plot(cm_right[:,0])
ax[1].plot(cm_left[:,0])

ax[2].plot(cm_right[:,1])
ax[2].plot(cm_left[:,1])

In [ ]:
left_hemisphere = frames[:,:,:160]
right_hemisphere = frames[:,:,160:]
cm_left = np.array([center_of_mass(f) for f in left_hemisphere])
cm_right = np.array([center_of_mass(f) for f in right_hemisphere])

fig, ax = plt.subplots(3)
fig.suptitle("Center of mass (contrast expected image)")

ax[0].plot(np.sum(difference, axis=(1,2)))

ax[1].plot(cm_right[:,0]-np.mean(cm_right[:,0]))
ax[1].plot(cm_left[:,0]-np.mean(cm_left[:,0]))

ax[2].plot(cm_right[:,1])
ax[2].plot(cm_left[:,1])

In [ ]:
%%capture
ani = show_video(normalize(max_filtered[600:]), normalize(difference[600:]), n_frames=200, vmin = .3, vmax=.7)

In [ ]:
# use median!